# Line detection

In [4]:
import json
import collections
import numpy as np

from datetime import datetime
from kafka import KafkaConsumer

def get_line_weights(current_players, lines, goalkeeper):
    line_weights = []
    current_players_wo_goalkeeper = current_players
    for idx, player in enumerate(current_players):
        if player == goalkeeper:
            #current_players_wo_goalkeeper.remove(goalkeeper)
            current_players_wo_goalkeeper.pop(idx)
    for line in lines:
        line_weights.append(np.sum([1 for player in current_players_wo_goalkeeper if player in line])/len(current_players_wo_goalkeeper))
    return line_weights

consumer = KafkaConsumer('playerPositionGroupedByTime', bootstrap_servers='86.119.35.55', auto_offset_reset='earliest')

last_players_team_0 = []
last_players_team_1 = []
last_line_team_0 = 0
last_line_team_1 = 0
t0_goalkeeper = 30
t0_line1 = [6, 11, 12, 24, 99]
t0_line2 = [8, 14, 19, 21, 23]
t0_line3 = [2, 5, 9, 17, 37]
t0_lines = [t0_line1, t0_line2, t0_line3]
t1_goalkeeper = 67
t1_line1 = [3, 8, 15, 16, 68]
t1_line2 = [13, 14, 28, 81, 90]
t1_line3 = [4, 11, 18, 24, 73]
t1_lines = [t1_line1, t1_line2, t1_line3]
line_names = ["Line 1", "Line 2", "Line 3"]

for idx, msg in enumerate(consumer):
    
    if idx > 500:
        break
    
    eventExists = False
    msg_json = json.loads(msg.value)
    
    current_players_team_0 = [int(x) for x in list(msg_json["teams"]["team_0"].keys())]
    current_players_team_1 = [int(x) for x in list(msg_json["teams"]["team_1"].keys())]

   # Filter out junk data
    if msg_json["time"] == 0 or len(current_players_team_0) < 6 or len(current_players_team_1) < 6:
        continue
        
    # New player 
    for player in current_players_team_0:
        if player not in last_players_team_0:
            print(msg_json["time"], "Player", player, " has joined Team 0", msg.offset)
            eventExists = True
    for player in current_players_team_1:
        if player not in last_players_team_1:
            print(msg_json["time"], "Player", player, " has joined Team 1", msg.offset)    
            eventExists = True
    # Player has left the field
    for player in last_players_team_0:
        if player not in current_players_team_0:
            print(msg_json["time"], "Player", player, " has left Team 0", msg.offset)
            eventExists = True
    for player in last_players_team_1:
        if player not in current_players_team_1:
            print(msg_json["time"], "Player", player, " has left Team 1", msg.offset)
            eventExists = True

    # Line detection
    current_line_weights_team_0 = get_line_weights(current_players=current_players_team_0, lines=t0_lines, goalkeeper=t0_goalkeeper)
    current_line_weights_team_1 = get_line_weights(current_players=current_players_team_1, lines=t1_lines, goalkeeper=t1_goalkeeper)
    current_line_team_0 = np.argmax(current_line_weights_team_0)
    current_line_team_1 = np.argmax(current_line_weights_team_1)   
    if current_line_team_0 != last_line_team_0:
        print(msg_json["time"], "Team 0 has switched from", line_names[last_line_team_0], "to", line_names[current_line_team_0], msg.offset)
        eventExists = True
    if current_line_team_1 != last_line_team_1:
        print(msg_json["time"], "Team 1 has switched from", line_names[last_line_team_1], "to", line_names[current_line_team_1], msg.offset)
        eventExists = True        
    
    if eventExists:
        print("Players Team 0:", sorted(current_players_team_0))
        print("Players Team 1:", sorted(current_players_team_1))

    # Set variables for next iteration
    last_players_team_0 = current_players_team_0
    last_players_team_1 = current_players_team_1
    last_line_team_0 = current_line_team_0
    last_line_team_1 = current_line_team_1

2022-08-21 10:30:57.581+0100 Player 99  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 30  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 24  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 12  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 11  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 6  has joined Team 0 252311
2022-08-21 10:30:57.581+0100 Player 68  has joined Team 1 252311
2022-08-21 10:30:57.581+0100 Player 67  has joined Team 1 252311
2022-08-21 10:30:57.581+0100 Player 16  has joined Team 1 252311
2022-08-21 10:30:57.581+0100 Player 15  has joined Team 1 252311
2022-08-21 10:30:57.581+0100 Player 8  has joined Team 1 252311
2022-08-21 10:30:57.581+0100 Player 3  has joined Team 1 252311
Players Team 0: [6, 11, 12, 24, 99]
Players Team 1: [3, 8, 15, 16, 68]
2022-08-21 10:30:45.100+0100 Player 30  has joined Team 0 252387
2022-08-21 10:30:45.100+0100 Player 67  has joined Team 1 252387
Players Team 0: [6, 11